In [ ]:
!nvidia-smi

Wed May 19 10:24:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install torch==1.8.1+cu102 torchvision==0.9.1+cu102 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu102/torch1.8.1/index.html

In [ ]:
%cd /content
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir ./

In [ ]:
!pip install timm
!pip install albumentations==0.4.6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Train

In [ ]:
%cd /content/drive/MyDrive/Stage3_Detection/code/Swin-Transformer-Object-Detection-master

/content/drive/MyDrive/Stage3_Detection/code/Swin-Transformer-Object-Detection-master


In [ ]:
!pip install -r ./requirements.txt

In [ ]:
# For train
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

import torch
import gc

# For inferecne
import os
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [ ]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/swin/JC_cascade_mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')

PREFIX = '../../input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.seg_prefix = PREFIX  # for applying HTC

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'valid.json'
# cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = [(512, 512),(576, 576),(640, 640),(704, 704),(768, 768),
                                          (832, 832),(896, 896),(960, 960),(1024, 1024)]
cfg.data.test.pipeline[1]['flip'] = True


cfg.data.samples_per_gpu = 4  # batch_size

cfg.seed = 31
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade'

cfg.model.roi_head.bbox_head[0]['num_classes'] = 11
cfg.model.roi_head.bbox_head[1]['num_classes'] = 11
cfg.model.roi_head.bbox_head[2]['num_classes'] = 11
cfg.model.roi_head.mask_head.num_classes = 11

cfg.model.roi_head.bbox_head[0]['norm_cfg']['type'] = 'BN'
cfg.model.roi_head.bbox_head[1]['norm_cfg']['type'] = 'BN'
cfg.model.roi_head.bbox_head[2]['norm_cfg']['type'] = 'BN'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

cfg.runner.max_epochs = 50
cfg.log_config.interval = 300
cfg.checkpoint_config = dict(type='CheckpointHook', interval=1, max_keep_ckpts=20)

cfg.evaluation.metric = ['bbox']

In [ ]:
cfg.albu_train_transforms

In [ ]:
cfg.train_pipeline[3]  # AutoAugment

In [ ]:
cfg.lr_config

In [ ]:
model = build_detector(cfg.model)

In [ ]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=11.16s)
creating index...
index created!


In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
loading annotations into memory...


2021-05-19 10:55:18,788 - mmdet - INFO - load checkpoint from ../../input/data/pth/cascade_mask_rcnn_swin_tiny_patch4_window7.pth
2021-05-19 10:55:18,791 - mmdet - INFO - Use load_from_local loader


Done (t=4.01s)
creating index...
index created!


2021-05-19 10:55:26,182 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.0.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.0.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: c

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0


...

Loading and preparing results...
DONE (t=0.65s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.93s).
Accumulating evaluation results...


2021-05-19 20:39:47,102 - mmdet - INFO - Saving checkpoint at 50 epochs


DONE (t=0.66s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.351
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.451
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.359
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.104
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.342
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.598
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.472
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.472
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.472
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.207
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.473
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.712


2021-05-19 20:39:53,430 - mmdet - INFO - Epoch(val) [50][655]	bbox_mAP: 0.3510, bbox_mAP_50: 0.4510, bbox_mAP_75: 0.3590, bbox_mAP_s: 0.1040, bbox_mAP_m: 0.3420, bbox_mAP_l: 0.5980, bbox_mAP_copypaste: 0.351 0.451 0.359 0.104 0.342 0.598
2021-05-19 20:44:31,989 - mmdet - INFO - Epoch [51][300/655]	lr: 1.000e-06, eta: 1:27:38, time: 0.928, data_time: 0.102, memory: 13531, loss_rpn_cls: 0.0273, loss_rpn_bbox: 0.0251, s0.loss_cls: 0.1512, s0.acc: 94.7640, s0.loss_bbox: 0.2168, s0.loss_mask: 0.1874, s1.loss_cls: 0.0669, s1.acc: 95.4199, s1.loss_bbox: 0.0891, s1.loss_mask: 0.0821, s2.loss_cls: 0.0324, s2.acc: 95.5168, s2.loss_bbox: 0.0348, s2.loss_mask: 0.0369, loss: 0.9500


KeyboardInterrupt: ignored

# Inference  ( fp16 주석 처리!! )

In [ ]:
file_name = 'epoch_50.pth'

cfg.work_dir = './work_dirs/cascade'
cfg.model.train_cfg = None

checkpoint_path = os.path.join(cfg.work_dir, f'{file_name}')

In [ ]:
cfg.data.test

In [ ]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

Use load_from_local loader


In [ ]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 0.6 task/s, elapsed: 1430s, ETA:     0s

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
output[0][0]

In [ ]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[0][j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{file_name}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.56805867 165.21016 76.83439 254.77577 166....,batch_01_vt/0021.jpg
1,1 0.2679668 285.71365 342.5805 306.80557 376.2...,batch_01_vt/0028.jpg
2,1 0.99610996 69.72126 168.78061 374.8547 378.8...,batch_01_vt/0031.jpg
3,1 0.06320084 295.4979 157.94702 362.29187 227....,batch_01_vt/0032.jpg
4,1 0.60295254 389.0904 271.01416 483.7361 344.8...,batch_01_vt/0070.jpg
